# Data Extraction

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline  
import tweepy
import csv

In [3]:
# Authentication and setup for Tweepy

consumer_key = 'aQzHCKeyTaBKwqJaaiwda4bkO'
consumer_secret = 'JLRfDVG7k8GbbkkOZofL6Al69P2Ov1xhWRBAjFNFR4FwSKoXDj'
access_token = '3125452793-u04kuuehiQIPh94QUelRV6RREMfOxoDBq7oznT7'
access_token_secret = 'xcejx5JhpsvRvsWylOjXbeKyUce8AjxgsmFz54v5qdvrh'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True )

In [100]:
handles = pd.read_csv('TwitterHandles.csv')

In [5]:
# Function which, given a set of tweets from Tweepy API, will write them to a csv file

def write_tweets_to_csv(tweets, csvWriter, political_affiliation):
    
    for tweet in tweets:
        csvWriter.writerow([tweet.user.screen_name, tweet.full_text.encode('utf-8'), political_affiliation])
        

In [6]:
# Function for retrieving latest tweets of a given username

def get_user_tweets(username, tweet_count = 200): 
    
    tweets = api.user_timeline(screen_name = username, count = tweet_count, 
                          include_rts = False, tweet_mode='extended')
    return tweets
    

In [7]:
# Main function for recording tweets from an account

def record_tweets(username, csvWriter, politic):
    
    tweets = get_user_tweets(username)
    
    write_tweets_to_csv(tweets, csvWriter, politic)
    

In [8]:
# Given a dataframe of politician usernames will c

def get_tweets_from_dataframe(dataframe, csvWriter):
    
    for index, row in dataframe.iterrows():
        
        record_tweets(row['TwitterHandle'], csvWriter, row['Party'])

In [5]:
# Initialize a csv file
csvFile = open('tweets_final3.csv', 'a')

#Use csv Writer
csvWriter = csv.writer(csvFile)

In [ ]:
get_tweets_from_dataframe(handles, csvWriter)

In [6]:
# Load the csv into a dataframe

tweets_df = pd.read_csv('tweets_final3.csv', names = ["username", "tweets_raw", "politics"])

# Data Cleaning

Consist mostly of processing the raw tweets to be more suitable as input features for LDA/sentiment extraction. Involves removing emoji encodings, special characters and the url to the tweet which tweepy embeds in every tweet

In [349]:
# Various methods for cleaning tweet strings to make them better suited for sentiment analysis

def remove_emojis(string):
    
    corrected = re.sub(r'\\x[\w+]{2}', '', string)
    
    return corrected

def remove_url(text):
    
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    
    return text

def correct_apostrophes(string):
    
    corrected = re.sub(r'\\xe2\\x80\\x99', '\'', string)
    
    corrected = re.sub(r'\&amp;', 'and', string)
    
    return corrected

def remove_byte_encoding(string):
    return string[2:]

def clean_tweet(tweet):
    
            
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet)

In [350]:
def clean_tweet(tweet):
    
        parts = tweet.split('.')
        
        for i in range(len(parts)):
            parts[i] = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", parts[i])
            
        cleaned_tweet = " ".join(parts)
        
        return cleaned_tweet

In [351]:
def clean_string(string):
    
    temp = remove_byte_encoding(string)
    temp = remove_url(temp)
    temp = correct_apostrophes(temp)
    temp = remove_emojis(temp)
    final = clean_tweet(temp)
    
    return final

In [352]:
def clean_dataframe_tweets(dataframe):
    
    new_strings = []
    
    for index, row in dataframe.iterrows():
        new_strings.append(clean_string(row['tweets_raw']))
    
    return new_strings

In [110]:
cleaned = clean_dataframe_tweets(tweets_df)

In [353]:
tweets_df['tweets_clean'] = cleaned

# Feature Engineering

### Sentiments, hashtags, and mentions

In [13]:
from textblob import TextBlob

#### Extracting hashtags and adding them as a feature

In [14]:
# Extracts all hashtags from a string. Useful for getting hashtags from tweets

def extract_hashtags(string):
    hashtags = re.findall(r"#(\w+)", string)
    
    return hashtags


# Iterates over a dataframe and extracts all hashtags

def find_hashtags(dataframe):
    
    h_list = []
    
    for index, row in dataframe.iterrows():
        
        hashtags = extract_hashtags(row['tweets_raw'])
        
        h_list.append(hashtags)
    
    return h_list

#### Extracting mentions and adding  them as a feature

In [15]:
# Extracts all hashtags from a string. Useful for getting hashtags from tweets

def extract_mentions(string):
    hashtags = re.findall(r"@(\w+)", string)
    
    return hashtags


# Iterates over a dataframe and extracts all hashtags

def find_mentions(dataframe):
    
    h_list = []
    
    for index, row in dataframe.iterrows():
        
        hashtags = extract_mentions(row['tweets_raw'])
        
        h_list.append(hashtags)
    
    return h_list

In [16]:
# Extract hashtags and add them as a separate column

hashtags = find_hashtags(tweets_df)

tweets_df['hashtags'] = hashtags

In [17]:
# Extract mentions and add them as a separate column

mentions = find_mentions(tweets_df)

tweets_df['mentions'] = mentions

## Sentiment extraction

In [18]:
def get_sentiment(string):
    text = TextBlob(string)
    
    return text.sentiment

def get_dataframe_sentiments(dataframe):
    
    sentiments = []
    
    for index, row in dataframe.iterrows():
        sentiments.append(get_sentiment(row['tweets_clean']))
        
    return sentiments

In [19]:
sentiments = get_dataframe_sentiments(tweets_df)

In [20]:
tweets_df['sentiment'] = sentiments

In [22]:
tweets_df_d = tweets_df.head(n=33099)

In [23]:
tweets_df_r = tweets_df.iloc[33099:]

In [57]:
# Construct corpus from the tweets, grouped by politics

In [24]:
s = tweets_df['tweets_clean']
s.str.cat(sep='. ')

corpus = s

In [25]:
s = tweets_df_d['tweets_clean']
s.str.cat(sep='. ')

corpus_d = s

In [26]:
s = tweets_df_r['tweets_clean']

corpus_r = s.str.cat(sep='. ')

### LDA

In [27]:
from nltk import FreqDist
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora
import string

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

C:\Users\PSpankay\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [29]:
doc_clean = [clean(doc).split() for doc in tweets_df_r['tweets_clean']]  
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [81]:
# Creating the object for LDA model
Lda = gensim.models.ldamodel.LdaModel

In [82]:
# Running and Trainign LDA model on the document term matrix
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=1)

In [30]:
def generate_ldaModels(floor, ceiling):
    
    if floor <= ceiling:
        
        models = []
        
        i = floor

        while i != ceiling:
            models.append(Lda(doc_term_matrix, num_topics=i, id2word = dictionary, passes=2))
            
            i += 1
        
        return models
    

In [31]:
def parse_topics(ldamodel):
    
    topics = []
    
    for i in range(len(ldamodel.print_topics())):
        
        values = []
        
        topic_words = ldamodel.print_topic(i).split('+')
        
        for i in range(len(topic_words)):
        
            values.append(get_values(topic_words[i], (i==len(topic_words)-1)))
        
        topics.append(values)
        
    return topics

In [32]:
def get_values(string, last=False):
    
    probability = string[0:6]
    
    if probability[-1] == '*':
        probability = probability[:-1]
        
    probability = float(probability)
    
    if last:
        word = string[7:-1]
    else: word = string[7:-2]
    
    return (probability, word)

In [28]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')

en_stop = get_stop_words('en')

#### Converting corpus to bag of words so it can be used for generating an LDA 

In [33]:
def preprocess_corpus(corpus):
    
    doc_clean = [clean(doc).split() for doc in corpus]  
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    
    return doc_term_matrix, dictionary

In [213]:
corpus_new, doc = preprocess_corpus(tweets_df_d['tweets_clean'])

#### Generating actual LDA model

In [358]:
def generate_LDA_model(corpus, num_topics=10):
    
    new_corpus, dictionary = preprocess_corpus(corpus)
    
    ldamodel = gensim.models.ldamodel.LdaModel(new_corpus, num_topics=num_topics, id2word = dictionary, passes=2)
    
    return ldamodel
    
    

In [35]:
def dist_to_row(dist, num_topics=10):
    
    row = []
    
    for i in range(num_topics):
        row.append(0)
    
    for topic in dist:
        row[topic[0]] = topic[1]
    
    return row

In [36]:

def get_topic_distribution(string, ldamodel):
    
    tokens = tokenizer.tokenize(string)
    
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    
    bow = ldamodel.id2word.doc2bow(texts)
    
    return ldamodel.get_document_topics(bow)
    

In [37]:
topic_column_names1 = ['1 - topic 0', '1 - topic 1', '1 - topic 0', '1 - topic 3',
                         '1 - topic 4', '1 - topic 5', '1 - topic 6', '1 - topic 7',
                         '1 - topic 8', '1 - topic 9']

topic_column_names2 = ['2 - topic 0', '2 - topic 1', '2- topic 0', '2- topic 3',
                         '2 - topic 4', '2 - topic 5', '2- topic 6', '2- topic 7',
                         '2 - topic 8', '2- topic 9']

In [274]:
d = pd.DataFrame(columns=topic_column_names1 + topic_column_names2)

In [38]:
def tweet_to_tdistrib(tweet, ldamodel, second=False):
    t_distrib = get_topic_distribution(tweet, ldamodel)
    
    if second:
        return pd.Series(dist_to_row(t_distrib), index = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19'])
    
    return pd.Series(dist_to_row(t_distrib))

In [39]:
def get_topic_features(dataframe, ldamodel1, ldamodel2):
    
    df = pd.DataFrame()
    
    for index, row in dataframe.iterrows():
        
        tweet = row['tweets_clean']
        
        new_row = tweet_to_row(tweet, ldamodel1, ldamodel2)
        
        df = df.append(new_row, ignore_index=True)

    return df
        

#### Making predictor using two LDA models 
#### one trained on republican tweets and one trained on democratic tweets

In [40]:
def tweet_to_row(tweet, lda1, lda2):
    
    t_row_1 = tweet_to_tdistrib(tweet, lda1)
    t_row_2 = tweet_to_tdistrib(tweet, lda2, True)
    
    new_row = t_row_1.append(t_row_2)
    
    return new_row

In [42]:
d_model = generate_LDA_model(tweets_df_d)

In [43]:
r_model = generate_LDA_model(tweets_df_r)

In [44]:
df_topics = get_topic_features(tweets_df, d_model, r_model)

In [45]:
df_topics.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19']

In [46]:
mapping = {
           'Democrat': 0,
           'Republican': 1}

In [47]:
def apply_sentiment(dataframe, sentiment):
    df = dataframe.mul(sentiment, axis=0)
    return df

In [48]:
def get_sentiments(dataframe):
    
    s = []
    
    for index, row in dataframe.iterrows():
        s.append(row['sentiment'][0])
    
    return s

In [49]:
sentiments = get_sentiments(tweets_df)

In [50]:
tweets_df['sentiment'] = sentiments

In [52]:
def gather_mentions(df):
    
    mentions = {}
    
    for index, row in df.iterrows():
        
        m = row['mentions']
        
        for mention in m:
            
            if mention not in mentions:
                mentions[mention] = 1
            
            else:
                mentions[mention] += 1
        
    return mentions
    

In [53]:
mlist = gather_mentions(tweets_df)

12054

#### See what the frequency of different users being mentioned is. Most  are mentioned only once

In [54]:
mdf = pd.DataFrame.from_records(mlist, index=[0])

In [58]:
mdf1 = mdf.transpose()

In [120]:
mdf1[0].value_counts()

1       8259
2       1654
3        644
4        337
5        234
6        150
7         97
8         89
9         70
10        63
12        57
11        42
13        38
14        36
15        23
16        21
17        14
18        13
19        11
28        10
20         9
21         9
30         8
23         8
22         7
33         7
25         6
24         6
27         6
32         5
        ... 
53         1
95         1
87         1
71         1
47         1
1166       1
214        1
86         1
70         1
589        1
109        1
93         1
69         1
61         1
1180       1
82         1
124        1
108        1
76         1
68         1
60         1
52         1
151        1
99         1
51         1
35         1
202        1
178        1
106        1
123        1
Name: 0, dtype: int64

#### Find all users who were mentioned at least 30 times in the corpus

In [126]:
mdf2 = mdf1[mdf1[0] >= 30]

In [127]:
mdf2.head()

,0
AIPAC,36
AjitPaiFCC,43
BarackObama,40
BetsyDeVosED,74
CDCgov,30


In [128]:
mdf3 = mdf2.transpose()

In [123]:
c = mdf3.columns

Index(['AIPAC', 'AjitPaiFCC', 'BarackObama', 'BetsyDeVosED', 'CFPB', 'CNN',
       'CongressionalAC', 'DHSgov', 'DeptVetAffairs', 'DeptofDefense',
       ...
       'louiseslaughter', 'marcorubio', 'nytimes', 'realDonaldTrump',
       'realdonaldtrump', 'repjohnlewis', 'the_USO', 'thehill', 'uscapitol',
       'usedgov'],
      dtype='object', length=111)

### Initial testing and model validation

In [112]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from matplotlib import pyplot as plt

In [113]:
topics_d = df_topics.loc[:, '0':'9']
topics_r = df_topics.loc[:, '10':'19']

topics_d = topics_d.astype('float')
topics_r = topics_r.astype('float')

In [114]:
d_final = apply_sentiment(topics_d, sentiments)
r_final = apply_sentiment(topics_r, sentiments)

In [492]:
dx = model.predict_proba(X)

#### Define the X and y matrices
Here X is each tweet's probability distribution over 10 different topics, and y is the political orientation of the politician who made the tweet

This is where I got my results for the initial round of testing

In [487]:
X = topics_d
y = tweets_df['politics'].map(mapping)

In [488]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [489]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [490]:
model.score(X,y)

0.5031544793606921

In [101]:
handles.shape

(436, 2)

### Mentions feature

In [129]:
mdf3.head()

,AIPAC,AjitPaiFCC,BarackObama,BetsyDeVosED,CDCgov,CFPB,CNN,ConawayTX11,CongressionalAC,DHSgov,...,realdonaldtrump,rep_stevewomack,repjohnlewis,the_USO,thehill,usairforce,uscapitol,usedgov,virginiafoxx,washingtonpost
0,36,43,40,74,30,55,68,30,48,74,...,106,30,49,59,55,32,85,47,34,33


In [130]:
mentiondf = pd.DataFrame(columns=mdf3.columns)

In [134]:
mentions_hash = {}

v = mentiondf.columns.values

for i in range(len(v)):
    mentions_hash[v[i]] = i
    

In [263]:
def getMentionVector(mentions, vlen):
    
    vector = [0]*vlen
    
    for m in mentions:
        
        if m in mentions_hash:
            
            index = mentions_hash[m]
            vector[index] = 1
    
    return pd.DataFrame(pd.Series(vector, index=mdf3.columns.values))

In [279]:
def getMentionVector(mentions, vlen):
    
    vector = [0]*vlen
    
    for m in mentions:
        
        if m in mentions_hash:
            
            index = mentions_hash[m]
            vector[index] = 1
    
    return vector

#### Function for constructing mention matrix
Feature I was thinking about but did not get to implement. The idea is building a co-occurence matrix for the usernames which were mentioned the 

In [281]:
def mentionMatrix(dataframe):
    
    rows = []
    
    i = 0
    
    while i < dataframe.shape[0]:
        
        new_row = getMentionVector(dataframe['mentions'][i], len(mdf3.columns.values))
        
        rows.append(new_row)
        
        i += 1
    
    df = pd.DataFrame(rows, columns = mdf3.columns.values)
    
    return df
        

In [189]:
tweets_df.shape

(66572, 7)

In [287]:
import timeit
start_time = timeit.default_timer()
mm = mentionMatrix(tweets_df)
elapsed = timeit.default_timer() - start_time
elapsed

3.290910984685979

In [212]:
%time
mm = mentionMatrix(tweets_df.iloc[0:1000])

Wall time: 0 ns


In [275]:
mn = mentionMatrix(tweets_df.iloc[0:2])

Empty DataFrame
Columns: [AIPAC, AjitPaiFCC, BarackObama, BetsyDeVosED, CDCgov, CFPB, CNN, ConawayTX11, CongressionalAC, DHSgov, DeptVetAffairs, DeptofDefense, DrPhilRoe, ENERGY, EPA, EPAScottPruitt, EPAoig, EdWorkforce, EmmanuelMacron, EnergyCommerce, FBI, FCC, FEMA, FLOTUS, FinancialCmte, FoxBusiness, FoxNews, GOP, GOPLeader, GOPoversight, Google, HASCDemocrats, HASCRepublicans, HHSGov, HUDgov, HispanicCaucus, HouseAgNews, HouseAppropsGOP, HouseCommerce, HouseDemocrats, HouseFloor, HouseForeign, HouseGOP, HouseHomeland, HouseJudDems, HouseJudiciary, HouseScience, HouseVetAffairs, HurdOnTheHill, IngrahamAngle, Interior, MSNBC, MacTXPress, MariaBartiromo, MickMulvaneyOMB, MorningsMaria, MrsB1stGraders, NASA, NIH, NRA, NYGovCuomo, NancyPelosi, NatResources, NatlParkService, OfficialCBC, POTUS, PPFA, PeteSessions, RealDonaldTrump, RepBarbaraLee, RepCurbelo, RepEspaillat, RepGoodlatte, RepKevinBrady, RepMarcyKaptur, RepMarkMeadows, RepMarkWalker, RepMcCaul, RepTedDeutch, RosLehtinen, SBAg

C:\Users\PSpankay\Anaconda3\lib\site-packages\pandas\indexes\api.py:71: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [237]:
mx = pd.concat(r, axis=1, keys=mdf3.columns.values)

In [305]:
mx = apply_sentiment(mm, sentiments)

Second round of testing using the sparse mentions matrix as a feature

In [309]:
X = mm
y = tweets_df['politics'].map(mapping)

In [310]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [315]:
model.score(X,y)

0.5411584449918885

In [312]:
my = topics_d.join(mm)

In [313]:
mys = apply_sentiment(my, sentiments)

In [314]:
X = mys
y = tweets_df['politics'].map(mapping)

model.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [319]:
a = tweets_df['username'].value_counts()

In [320]:
len(a)

427

### Aggregate tweets by author

In [431]:
def aggregateTweets(dataframe):
    
    t = []
    
    current = ''
    
    i = -1
    
    for index, row in dataframe.iterrows():
        
        if current != row['username']:
            
            i += 1
            
            current = row['username']
            
            t.append([row['username'], row['tweets_raw'], row['tweets_clean'],  row['politics']])
            
    
        else:
            t[i][1] += " " + row['tweets_clean']
    
    
    return t
    

In [434]:
tdic = aggregateTweets(tweets_df)

In [435]:
author_df = pd.DataFrame(tdic, columns = ['username', 'tweets_raw', 'tweets_clean', 'politic'])

In [436]:
author_df.head()

,username,tweets_raw,tweets_clean,politic
0,RepDarrenSoto,b'Touching &amp; productive conversation w/ Fl...,Touching and productive conversation w Florid...,Democrat
1,RepJackyRosen,"b""Today, I met with the Nevada Radiological So...",Today I met with the Nevada Radiological Soci...,Democrat
2,RepAlLawsonJr,b'Looking forward to speaking on the House Flo...,Looking forward to speaking on the House Floor...,Democrat
3,RepEspaillat,b'Photos from today\xe2\x80\x99s reception in ...,Photos from todays reception in Washington DC ...,Democrat
4,RepBRochester,"b'Today, I took part in a gun violence prevent...",Today I took part in a gun violence preventio...,Democrat


In [359]:
newModel = generate_LDA_model(author_df['tweets_clean'])

In [364]:
def get_topic_features1(dataframe, ldamodel):
    
    df = pd.DataFrame()
    
    for index, row in dataframe.iterrows():
        
        tweet = row['tweets_clean']
        
        new_row = tweet_to_row1(tweet, ldamodel)
        
        df = df.append(new_row, ignore_index=True)

    return df
        
def tweet_to_row1(tweet, lda1):
    
    t_row_1 = tweet_to_tdistrib(tweet, lda1)
    
    new_row = t_row_1
    
    return new_row

def tweet_to_tdistrib(tweet, ldamodel, second=False):
    t_distrib = get_topic_distribution(tweet, ldamodel)
    
    if second:
        return pd.Series(dist_to_row(t_distrib), index = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19'])
    
    return pd.Series(dist_to_row(t_distrib))

In [505]:
author_topics = get_topic_features1(author_df, newModel)

In [367]:
author_topics.head()

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.0,0.0,0.330500,0.405258,0.000000,0.045051,0.082600,0.135991,0.0
1,0.000000,0.0,0.0,0.000000,0.581370,0.000000,0.338031,0.066092,0.000000,0.0
2,0.000000,0.0,0.0,0.017172,0.169129,0.540331,0.022605,0.250519,0.000000,0.0
3,0.000000,0.0,0.0,0.000000,0.269903,0.000000,0.670603,0.043834,0.000000,0.0
4,0.021923,0.0,0.0,0.016172,0.678088,0.082257,0.000000,0.180526,0.000000,0.0


#### Third round of testing
This is where I tested my model on collections of author's tweets as opposed to feeding in tweets individually

In [392]:
X = author_topics
y = author_df['politic'].map(mapping)

In [406]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [407]:
# instantiate a logistic regression model, and fit with X and y

model = LogisticRegression()

model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [405]:
model.score(X_test, y_test)

0.8488372093023255

In [423]:
author_topics_s.shape

(427, 10)

In [497]:
author_df.head()

,username,tweets_raw,tweets_clean,politic,mentions,mmentions
0,RepDarrenSoto,b'Touching &amp; productive conversation w/ Fl...,Touching and productive conversation w Florid...,Democrat,[],
1,RepJackyRosen,"b""Today, I met with the Nevada Radiological So...",Today I met with the Nevada Radiological Soci...,Democrat,[],
2,RepAlLawsonJr,b'Looking forward to speaking on the House Flo...,Looking forward to speaking on the House Floor...,Democrat,[],
3,RepEspaillat,b'Photos from today\xe2\x80\x99s reception in ...,Photos from todays reception in Washington DC ...,Democrat,[RepEspaillat],RepEspaillat
4,RepBRochester,"b'Today, I took part in a gun violence prevent...",Today I took part in a gun violence preventio...,Democrat,[],


In [498]:
author_df_d = author_df.loc[author_df['politic']=='Democrat']
author_df_r = author_df.loc[author_df['politic']=='Republican']

In [510]:
model_d =generate_LDA_model(author_df_d['tweets_clean'])
model_r =generate_LDA_model(author_df_r['tweets_clean'])

In [514]:
author_topicsd = get_topic_features1(author_df_d, model_d)
author_topicsr = get_topic_features1(author_df_r, model_r)

In [513]:
def get_probs(X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
    model= LogisticRegression()

    model.fit(X_train,y_train)
    
    probs = model.predict_proba(X_test)
    
    return probs

,0,1,2,3,4,5,6,7,8,9
0,0.00000,0.000000,0.772162,0.140773,0.000000,0.0,0.00000,0.052060,0.000000,0.0
1,0.00000,0.000000,0.850000,0.000000,0.000000,0.0,0.00000,0.000000,0.088455,0.0
2,0.84568,0.000000,0.000000,0.000000,0.000000,0.0,0.09717,0.000000,0.000000,0.0
3,0.00000,0.000000,0.233870,0.000000,0.704582,0.0,0.00000,0.000000,0.000000,0.0
4,0.00000,0.051924,0.000000,0.000000,0.000000,0.0,0.00000,0.754004,0.164902,0.0


In [515]:
probs = get_probs(author_topicsd, 

NameError: name 'probs' is not defined

In [ ]:
def predict_two_probs(probs1, probs2):
    
    predicts = []
    
    for i in range(len(predicts)):
        p0 = probs1[i]

In [426]:
sentiments = get_dataframe_sentiments(author_df)

In [421]:
len(sentiments)

66572

In [427]:
author_topics['sentiment'] = sentiments

In [532]:
tweets_df.head()

,username,tweets_raw,politics,tweets_clean,hashtags,mentions,sentiment
0,RepDarrenSoto,b'Touching &amp; productive conversation w/ Fl...,Democrat,Touching and productive conversation w Florid...,[NeverAgain],[],0.233333
1,RepDarrenSoto,"b""Great news! Central FL's @OrangeCoSheriff aw...",Democrat,Great news Central FL s awarded over 1 mil...,[Sayfie],"[OrangeCoSheriff, fema]",0.341667
2,RepDarrenSoto,b'2016 Fla elections were subject to cyberatta...,Democrat,2016 Fla elections were subject to cyberattack...,[Sayfie],"[HispanicCaucus, HouseNewDems]",-0.083333
3,RepDarrenSoto,b'Enjoyed assembling snack packages with colle...,Democrat,Enjoyed assembling snack packages with colleag...,[BeTheForce],[the_USO],0.350000
4,RepDarrenSoto,b'Puerto Rico needs a long-term solution! CMS ...,Democrat,Puerto Rico needs a long term solution CMS mu...,[HurricaneSeason2018],"[Pwr4PuertoRico, HispanicCaucus, HispanicFed, ...",0.225000


In [443]:

mentions = find_mentions(author_df)

author_df['mentions'] = mentions
mm = mentionMatrix(author_df)

df_final = author_topics.join(mm)

In [465]:
def merge_mentions(df):
    
    merged_mentions = []
    
    for index, row in df.iterrows():
        
        mm = ' '.join(row['mentions'])
        
        merged_mentions.append(mm)
        
    
    return merged_mentions
        

In [466]:
mmentions = merge_mentions(author_df)

In [467]:
author_df['mmentions'] = mmentions

In [445]:
X = df_final
y = author_df['politic']

In [446]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [447]:
model = LogisticRegression()

model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [449]:
model.score(X_test, y_test)

0.8488372093023255

In [450]:
df_final.shape

(427, 147)

In [452]:
mdf3.head()

,AIPAC,AjitPaiFCC,BarackObama,BetsyDeVosED,CDCgov,CFPB,CNN,ConawayTX11,CongressionalAC,DHSgov,...,realdonaldtrump,rep_stevewomack,repjohnlewis,the_USO,thehill,usairforce,uscapitol,usedgov,virginiafoxx,washingtonpost
0,36,43,40,74,30,55,68,30,48,74,...,106,30,49,59,55,32,85,47,34,33


In [453]:
new_columns = mdf3.columns[mdf3.ix[mdf3.last_valid_index()].argsort()]

In [454]:
mdf3[new_columns]

,SenJohnMcCain,RepMarkMeadows,dcexaminer,ConawayTX11,Google,mail,rep_stevewomack,CDCgov,USArmyReserve,EPAoig,...,FCC,FoxBusiness,EPA,HouseCommerce,FoxNews,EPAScottPruitt,SpeakerRyan,HouseGOP,POTUS,realDonaldTrump
0,30,30,30,30,30,30,30,30,31,31,...,168,178,202,214,232,295,488,589,1166,1180


In [482]:
cc_matrix = pd.DataFrame(0, index = mdf3.columns, columns = mdf3.columns, )

In [484]:
cc_matrix.head()

,AIPAC,AjitPaiFCC,BarackObama,BetsyDeVosED,CDCgov,CFPB,CNN,ConawayTX11,CongressionalAC,DHSgov,...,realdonaldtrump,rep_stevewomack,repjohnlewis,the_USO,thehill,usairforce,uscapitol,usedgov,virginiafoxx,washingtonpost
AIPAC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AjitPaiFCC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BarackObama,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BetsyDeVosED,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDCgov,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [486]:
cc_matrix['AIPAC']['CDCgov']

0

In [518]:
author_topics[5][2]

0.37066400051116943

In [528]:
def get_hashtag_sentiment(hashtag, df):
    
    s = [0]*df.shape[0]
    
    index = 0
    
    current = ''
    
    i = 0 
    
    while i < df.shape[0]:
        
        if hashtag in df['mentions'][i]:
            
            if df['username'][i] != current:
                current = df['mentions'][i]
                
                s.append(get_sentiment(df['tweets_clean'][i]))
            
            else:
                s[len(s) - 1] += get_sentiment(df['tweets_clean'][i])
        
        i += 1
    
    return s
                
        

In [549]:
def get_hashtag_sentiment(hashtag, df):
    
    current = ''
    s = []
    
    i = 0
    
    while i < df.shape[0]:
        
        if df['username'][i] != current:
            s.append(0)
            
            current = df['username'][i]
            
        if hashtag in df['mentions'][i]:
            
            s[len(s) - 1] += get_sentiment(df['tweets_clean'][i])[0]
        
        i += 1
    
    return s
            
            

In [550]:
st = get_hashtag_sentiment('realDonaldTrump', tweets_df)

In [551]:
author_topics['rdt_sentiment'] = st

In [553]:
from sklearn.feature_extraction.text import CountVectorizer
    
def cc_matrix(wordlist):
    
    samples = wordlist
    bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), vocabulary = {'awesome unicorns':0, 'batman forever':1}) 
    co_occurrences = bigram_vectorizer.fit_transform(samples)
    print('Printing sparse matrix:' + str(co_occurrences))
    print('Printing dense matrix (cols are vocabulary keys 0-> "awesome unicorns", 1-> "batman forever")' +str(co_occurrences.todense()))
    sum_occ = np.sum(co_occurrences.todense(),axis=0)
    print ('Sum of word-word occurrences:' + str(sum_occ))
    print ('Pretty printig of co_occurrences count:', zip(bigram_vectorizer.get_feature_names(),np.array(sum_occ)[0].tolist()))

    
#cc_matrix(author_df['mmentions'])

In [554]:
X = author_topics
y = author_df['politic']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [555]:
model.score(X_test, y_test)

0.6355140186915887